In [1]:
# the packages
import pandas as pd
import numpy as np
import networkx as nx

# import my own helper functions
from read import read_sims_result
from clean import cleanup_0IR_exp
from clean import cleanup_network

# Page Rank
from networkx.algorithms.link_analysis.pagerank_alg import pagerank_numpy

# logistic regression
from sklearn.linear_model import LogisticRegression

In [2]:
# ###########################
# Read OIR results, and fit the model
# ###########################
df0 = read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/0622/0IR", 32)
df0c = cleanup_0IR_exp(df0, numNode=32, numPeriod=15, numSim=300, balanced=True)

X = df0c[["deposits", "cash", "assets", "credit available", "wealth", "leverage", 
         "dummy-0-leverage",
         "wealth-lag", "deposits-lag", "cash-lag", "assets-lag", "leverage-lag", 
         "credit-available-lag", "credit-issued-lag", "dummy-0-leverage-lag",
         "over-leverage-frequency"]]
y = df0c["default-next"]

final = LogisticRegression(penalty="l1", C=0.007)
final.fit(X,y)

LogisticRegression(C=0.007, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [4]:
# make sure no defaults in 0IR
# sum(df0[df0["defaults due to interest"]
#     +df0["defaults due to negative wealth"]
#     +df0["defaults due to deposit shock"] == 0].loc[:,"dot1":"dot31"].values)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

In [5]:
# ###########################
# Read & process positive IR results
# ###########################
df_1 = read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/0625/1IR", 32)
mx_1n = cleanup_network(df_1, numNode=32, numPeriod=15, numSim=50)
df_1c = cleanup_0IR_exp(df_1, numNode=32, numPeriod=15, numSim=50)

In [6]:
mx_1n.shape

(50, 13, 31, 31)

50*13*31

In [7]:
df_1c.size

437760

In [8]:
# ###########################
# Let's add the weight
# ###########################

In [9]:
opq = np.array([2, 5 ,6])
opq * 0.4

array([0.8, 2. , 2.4])

In [10]:
G = nx.DiGraph(mx_1n[3,2])
nx.draw(G)

In [11]:
pg = pagerank_numpy(G)
for k,v in pg.items():
    print("{:<3}| {:.8}".format(k,v))

0  | 0.035259265
1  | 0.030456189
2  | 0.030456189
3  | 0.030456189
4  | 0.030456189
5  | 0.030456189
6  | 0.030456189
7  | 0.030456189
8  | 0.035648913
9  | 0.035380293
10 | 0.035119624
11 | 0.033111168
12 | 0.034646382
13 | 0.034436413
14 | 0.030456189
15 | 0.030456189
16 | 0.03390325
17 | 0.030456189
18 | 0.033614979
19 | 0.030456189
20 | 0.033156904
21 | 0.03297282
22 | 0.032811638
23 | 0.030456189
24 | 0.032543206
25 | 0.032430377
26 | 0.032328977
27 | 0.032237372
28 | 0.03140625
29 | 0.030456189
30 | 0.033061711
